In [14]:
import time
import sys
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split, KFold
import multiprocessing
import os
from tqdm import tqdm
from scipy import sparse

In [15]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from scipy.stats import spearmanr, pearsonr
from sklearn import ensemble

In [16]:
import datetime as dt
from datetime import date
import calendar
from calendar import weekday, day_name

In [17]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import scipy as sp

In [12]:
weather_file = pd.read_csv("Data/weather_processed.csv", sep = ',')
#date와 station_nbr정보를 가져옴
multi_index = [weather_file.values[:,1], weather_file.values[:,0]]
#index를 date와 station_nbr기준으로 하여 데이터를 가져옴
df = pd.DataFrame(weather_file.values[:,2:], index=multi_index, columns=weather_file.columns.values[2:])
df

tmax tmin tavg depart dewpoint wetbulb heat cool sunrise sunset  \
2012-01-01 1    52   31   42     16       36      40   23    0     716   1626   
           2    48   33   41     16       37      39   24    0     716   1626   
           3    55   34   45      9       24      36   20    0     735   1720   
           4    63   47   55      4       28      43   10    0     728   1742   
           6    63   34   49      0       31      43   16    0     727   1742   
           7    50   33   42      0       26      35   23    0     727   1742   
           8    66   45   42      0       34      46   23    0     727   1742   
           9    34   19   27      0       17      23   38    0     727   1742   
           10   73   53   63      0       55      58    2    0     723   1738   
           11   72   48   60      7       54      56    5    0     724   1737   
           12   72   48   60      7       54      57    5    0     724   1737   
           13   38   18   28      7       12      21   37    0     724   1737   
           14   50   34   42      5       25      35   23    0     739   1729   
           15   48   26   37     16       35      38   28    0     718   1621   
           16   50   30   40     16       37      40   25    0     718   1621   
           17   56   38   47     16       23      37   18    0     718   1621   
           18   59   40   50      4       28      40   15    0     730   1732   
           19   38   25   32     10       26      30   33    0     723   1628   
           20   59   41   50     10       28      40   15    0     723   1628   
2012-01-02 1    50   31   41     10       26      35   24    0     723   1628   
           2    46   28   37     12       24      32   28    0     716   1627   
           3    45   24   35     -1       11      28   30    0     735   1721   
           4    60   37   49     -2       26      39   16    0     728   1742   
           6    60   30   45     -4       27      38   20    0     727   1742   
           7    44   26   35     -4       17      29   30    0     727   1742   
           8    59   36   35     -4       29      40   30    0     727   1742   
           9    21   11   16     -4        8      15   49    0     727   1742   
           10   65   45   55     -4       33      46   10    0     723   1738   
           11   63   40   52     -1       31      44   13    0     724   1738   
           12   65   47   56     -1       33      46    9    0     724   1738   
...            ...  ...  ...    ...      ...     ...  ...  ...     ...    ...   
2014-10-30 11   81   59   70      4       63      65    0    5     640   1741   
           12   81   61   71      4       63      66    0    6     640   1741   
           13   61   32   47      4       26      38   18    0     640   1741   
           14   77   45   61      5       39      50    4    0     650   1738   
           15   56   30   43      0       36      39   22    0     618   1641   
           16   57   34   46      0       39      43   19    0     618   1641   
           17   77   43   60      0       41      50    5    0     618   1641   
           18   79   56   68      5       51      58    0    3     643   1739   
           19   51   32   42     -4       33      40   23    0     623   1647   
           20   80   52   66     -4       50      57    0    1     623   1647   
2014-10-31 1    53   40   47     -4       36      41   18    0     623   1647   
           2    49   40   45      0       35      40   20    0     619   1643   
           3    55   33   44    -12       32      41   21    0     645   1729   
           4    76   56   66     -1       47      56    0    1     644   1745   
           5    76   56   66     -1       37      56    0    1     616   1640   
           6    76   51   64      0       47      55    1    0     644   1745   
           7    58   38   48      0       32      41   17    0     644   1745   
           8    78   55   67      0       32 

In [14]:
key_file = pd.read_csv("Data/key.csv", sep=',')
key_file

store_nbr  station_nbr
0           1            1
1           2           14
2           3            7
3           4            9
4           5           12
5           6           14
6           7            6
7           8            4
8           9           17
9          10           12
10         11           10
11         12           11
12         13            6
13         14           16
14         15           13
15         16            2
16         17           20
17         18           17
18         19           15
19         20            7
20         21            3
21         22           10
22         23           17
23         24            9
24         25           13
25         26           17
26         27           10
27         28            7
28         29            3
29         30           19
30         31           17
31         32           13
32         33            3
33         34           17
34         35            5
35         36           18
36         37           13
37         38           14
38         39            8
39         40           13
40         41           12
41         42           14
42         43           11
43         44           12
44         45           16

In [15]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
 
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
 
    return False

In [21]:
token = ('+FCFCTSGRRADZSNSGGSPLICFG+FGBRUPHZFUVADUDSPOSOSASSPYSQDRSHFZMIPRBCBLVC')

In [19]:
# codesum 분류를 위한 기준 설정 및 추출 함수
ls = np.array(['+FC', 'FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG+', 
               'FG', 'BR', 'UP', 'HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SO', 'SA', 'SS',
                'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC'])



def validation(token):
    result3 = []
    for each in ls:
        if each in token:
            token = token.replace(each,'')
            result3.append(each)
    return result3


def extract_cs():
    test = list(weather.codesum.unique())
    result = []
    for each in test:
        result.append(validation(each))
    return result

In [22]:
weather = pd.read_csv("Data/weather.csv", sep=',')
# 위 codesum 추출 함수를 기반으로 codesum value에 해당하는 컬럼 및 값 부여
cs_iter = (set(validation(token)) for x in weather.codesum)
cs = sorted(set.union(*cs_iter))

dummies = pd.DataFrame(np.zeros((len(weather), len(cs))), columns = cs)


for i, gen in enumerate(weather.codesum):
    dummies.ix[i, validation(gen)] = '1'
    
weather_cs = weather.join(dummies)
weather_cs

/Users/woongkook/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.


station_nbr        date tmax tmin tavg depart dewpoint wetbulb heat  \
0                1  2012-01-01   52   31   42      M       36      40   23   
1                2  2012-01-01   48   33   41     16       37      39   24   
2                3  2012-01-01   55   34   45      9       24      36   20   
3                4  2012-01-01   63   47   55      4       28      43   10   
4                6  2012-01-01   63   34   49      0       31      43   16   
5                7  2012-01-01   50   33   42      M       26      35   23   
6                8  2012-01-01   66   45    M      M       34      46    M   
7                9  2012-01-01   34   19   27      M       17      23   38   
8               10  2012-01-01   73   53   63      M       55      58    2   
9               11  2012-01-01   72   48   60      7       54      56    5   
10              12  2012-01-01   72   48   60      M       54      57    5   
11              13  2012-01-01   38   18   28      M       12      21   37   
12              14  2012-01-01   50   34   42      5       25      35   23   
13              15  2012-01-01   48   26   37     16       35      38   28   
14              16  2012-01-01   50   30   40      M       37      40   25   
15              17  2012-01-01   56   38   47      M       23      37   18   
16              18  2012-01-01   59   40   50      4       28      40   15   
17              19  2012-01-01   38   25   32     10       26      30   33   
18              20  2012-01-01   59   41   50      M       28      40   15   
19               1  2012-01-02   50   31   41      M       26      35   24   
20               2  2012-01-02   46   28   37     12       24      32   28   
21               3  2012-01-02   45   24   35     -1       11      28   30   
22               4  2012-01-02   60   37   49     -2       26      39   16   
23               6  2012-01-02   60   30   45     -4       27      38   20   
24               7  2012-01-02   44   26   35      M       17      29   30   
25               8  2012-01-02   59   36    M      M       29      40    M   
26               9  2012-01-02   21   11   16      M        8      15   49   
27              10  2012-01-02   65   45   55      M       33      46   10   
28              11  2012-01-02   63   40   52     -1       31      44   13   
29              12  2012-01-02   65   47   56      M       33      46    9   
...            ...         ...  ...  ...  ...    ...      ...     ...  ...   
20487           11  2014-10-30   81   59   70      4       63      65    0   
20488           12  2014-10-30   81   61   71      M       63      66    0   
20489           13  2014-10-30   61   32   47      M       26      38   18   
20490           14  2014-10-30   77   45   61      5       39      50    4   
20491           15  2014-10-30   56   30   43      0       36      39   22   
20492           16  2014-10-30   57   34   46      M       39      43   19   
20493           17  2014-10-30   77   43   60      M       41      50    5   
20494           18  2014-10-30   79   56   68      5       51      58    0   
20495           19  2014-10-30   51   32   42     -4       33      40   23   
20496           20  2014-10-30   80   52   66      M       50      57    0   
20497            1  2014-10-31   53   40   47      M       36      41   18   
20498            2  2014-10-31   49   40   45      0       35      40   20   
20499            3  2014-10-31   55   33   44    -12       32      41   21   
20500            4  2014-10-31   76   56   66     -1       47      56    0   
20501            5  2014-10-31    M    M    M      M       37       M    M   
20502            6  2014-10-31   76   51   64      0       47      55    1   
20503            7  2014-10-31   58   38   48      M       32      41   17   
20504            8  2014-10-31   78   55   67      M        M       M    0   
20505            9  2014-10-31   39   25   32      M       18      27   33   
20506           10  2014-10-3

In [19]:
weather_file = pd.read_csv("Data/weather.csv", sep=',')
weather_data = weather_file.drop('codesum', axis = 1) # codesum 컬럼 삭제
    
data = weather_data.values[:,2:] # index, station_nbr, date를 제외한 나머지 데이터 array로 변경
# 데이터가 있을 경우 float형으로 변경, 데이터가 없을 경우 unicodedata로 변경
for col in range(data.shape[1]):
    for row in range(data.shape[0]):
        if not is_number(data[row, col]):
            if row == 0:
                i = 1
                while not is_number(data[row+i, col]):
                    i += 1
                data[row, col] = data[row+i, col]
            else:
                data[row, col] = data[row-1, col]
        data[row, col] = float(data[row, col]) # Convert string to float value

weather_processed = np.append(weather_file.values[:,0:2], data, axis = 1)
weather_df = pd.DataFrame(weather_processed, index=None, columns=weather_data.columns.values)
    
weather_df

station_nbr        date tmax tmin tavg depart dewpoint wetbulb heat  \
0               1  2012-01-01   52   31   42     16       36      40   23   
1               2  2012-01-01   48   33   41     16       37      39   24   
2               3  2012-01-01   55   34   45      9       24      36   20   
3               4  2012-01-01   63   47   55      4       28      43   10   
4               6  2012-01-01   63   34   49      0       31      43   16   
5               7  2012-01-01   50   33   42      0       26      35   23   
6               8  2012-01-01   66   45   42      0       34      46   23   
7               9  2012-01-01   34   19   27      0       17      23   38   
8              10  2012-01-01   73   53   63      0       55      58    2   
9              11  2012-01-01   72   48   60      7       54      56    5   
10             12  2012-01-01   72   48   60      7       54      57    5   
11             13  2012-01-01   38   18   28      7       12      21   37   
12             14  2012-01-01   50   34   42      5       25      35   23   
13             15  2012-01-01   48   26   37     16       35      38   28   
14             16  2012-01-01   50   30   40     16       37      40   25   
15             17  2012-01-01   56   38   47     16       23      37   18   
16             18  2012-01-01   59   40   50      4       28      40   15   
17             19  2012-01-01   38   25   32     10       26      30   33   
18             20  2012-01-01   59   41   50     10       28      40   15   
19              1  2012-01-02   50   31   41     10       26      35   24   
20              2  2012-01-02   46   28   37     12       24      32   28   
21              3  2012-01-02   45   24   35     -1       11      28   30   
22              4  2012-01-02   60   37   49     -2       26      39   16   
23              6  2012-01-02   60   30   45     -4       27      38   20   
24              7  2012-01-02   44   26   35     -4       17      29   30   
25              8  2012-01-02   59   36   35     -4       29      40   30   
26              9  2012-01-02   21   11   16     -4        8      15   49   
27             10  2012-01-02   65   45   55     -4       33      46   10   
28             11  2012-01-02   63   40   52     -1       31      44   13   
29             12  2012-01-02   65   47   56     -1       33      46    9   
...           ...         ...  ...  ...  ...    ...      ...     ...  ...   
20487          11  2014-10-30   81   59   70      4       63      65    0   
20488          12  2014-10-30   81   61   71      4       63      66    0   
20489          13  2014-10-30   61   32   47      4       26      38   18   
20490          14  2014-10-30   77   45   61      5       39      50    4   
20491          15  2014-10-30   56   30   43      0       36      39   22   
20492          16  2014-10-30   57   34   46      0       39      43   19   
20493          17  2014-10-30   77   43   60      0       41      50    5   
20494          18  2014-10-30   79   56   68      5       51      58    0   
20495          19  2014-10-30   51   32   42     -4       33      40   23   
20496          20  2014-10-30   80   52   66     -4       50      57    0   
20497           1  2014-10-31   53   40   47     -4       36      41   18   
20498           2  2014-10-31   49   40   45      0       35      40   20   
20499           3  2014-10-31   55   33   44    -12       32      41   21   
20500           4  2014-10-31   76   56   66     -1       47      56    0   
20501           5  2014-10-31   76   56   66     -1       37      56    0   
20502           6  2014-10-31   76   51   64      0       47      55    1   
20503           7  2014-10-31   58   38   48      0       32      41   17   
20504           8  2014-10-31   78   55   67      0       32      41    0   
20505           9  2014-10-31   39   25   32      0       18      27   33   
20506          10  2014-10-31   77   63   70      0       51      59    0   
20

In [20]:
def process_weather():
    weather_file = pd.read_csv("Data/weather.csv", sep=',')
    weather_data = weather_file.drop('codesum', axis = 1) # codesum 컬럼 삭제
    
    data = weather_data.values[:,2:] # index, station_nbr, date를 제외한 나머지 데이터 array로 변경
    # 데이터가 있을 경우 float형으로 변경, 데이터가 없을 경우 unicodedata로 변경
    for col in range(data.shape[1]):
        for row in range(data.shape[0]):
            if not is_number(data[row, col]):
                if row == 0:
                    i = 1
                    while not is_number(data[row+i, col]):
                        i += 1
                    data[row, col] = data[row+i, col]
                else:
                    data[row, col] = data[row-1, col]
            data[row, col] = float(data[row, col]) # Convert string to float value

    weather_processed = np.append(weather_file.values[:,0:2], data, axis = 1)
    df = pd.DataFrame(weather_processed, index=None, columns=weather_data.columns.values)
    df.to_csv("Data/weather_processed.csv", sep=',', index=None)

In [21]:
key_file = pd.read_csv("Data/key.csv", sep=',')
df = pd.DataFrame(key_file.values[:,1], index = key_file.values[:,0])
df
get_station_data()

NameError: name 'get_station_data' is not defined

In [22]:
#index를 제외한 key데이터를 dataframe로 가져옴
def match_store_station():
    key_file = pd.read_csv("Data/key.csv", sep=',')
    df = pd.DataFrame(key_file.values[:,1], index = key_file.values[:,0])
    return df

In [23]:
def read_processed_weather():
    weather_file = pd.read_csv("Data/weather_processed.csv", sep = ',')
    #date와 station_nbr정보를 가져옴
    multi_index = [weather_file.values[:,1], weather_file.values[:,0]]
    #index를 date와 station_nbr기준으로 하여 데이터를 가져옴
    df = pd.DataFrame(weather_file.values[:,2:], index=multi_index, columns=weather_file.columns.values[2:])
    return df

In [24]:
a = read_processed_weather()
a

tmax tmin tavg depart dewpoint wetbulb heat cool sunrise sunset  \
2012-01-01 1    52   31   42     16       36      40   23    0     716   1626   
           2    48   33   41     16       37      39   24    0     716   1626   
           3    55   34   45      9       24      36   20    0     735   1720   
           4    63   47   55      4       28      43   10    0     728   1742   
           6    63   34   49      0       31      43   16    0     727   1742   
           7    50   33   42      0       26      35   23    0     727   1742   
           8    66   45   42      0       34      46   23    0     727   1742   
           9    34   19   27      0       17      23   38    0     727   1742   
           10   73   53   63      0       55      58    2    0     723   1738   
           11   72   48   60      7       54      56    5    0     724   1737   
           12   72   48   60      7       54      57    5    0     724   1737   
           13   38   18   28      7       12      21   37    0     724   1737   
           14   50   34   42      5       25      35   23    0     739   1729   
           15   48   26   37     16       35      38   28    0     718   1621   
           16   50   30   40     16       37      40   25    0     718   1621   
           17   56   38   47     16       23      37   18    0     718   1621   
           18   59   40   50      4       28      40   15    0     730   1732   
           19   38   25   32     10       26      30   33    0     723   1628   
           20   59   41   50     10       28      40   15    0     723   1628   
2012-01-02 1    50   31   41     10       26      35   24    0     723   1628   
           2    46   28   37     12       24      32   28    0     716   1627   
           3    45   24   35     -1       11      28   30    0     735   1721   
           4    60   37   49     -2       26      39   16    0     728   1742   
           6    60   30   45     -4       27      38   20    0     727   1742   
           7    44   26   35     -4       17      29   30    0     727   1742   
           8    59   36   35     -4       29      40   30    0     727   1742   
           9    21   11   16     -4        8      15   49    0     727   1742   
           10   65   45   55     -4       33      46   10    0     723   1738   
           11   63   40   52     -1       31      44   13    0     724   1738   
           12   65   47   56     -1       33      46    9    0     724   1738   
...            ...  ...  ...    ...      ...     ...  ...  ...     ...    ...   
2014-10-30 11   81   59   70      4       63      65    0    5     640   1741   
           12   81   61   71      4       63      66    0    6     640   1741   
           13   61   32   47      4       26      38   18    0     640   1741   
           14   77   45   61      5       39      50    4    0     650   1738   
           15   56   30   43      0       36      39   22    0     618   1641   
           16   57   34   46      0       39      43   19    0     618   1641   
           17   77   43   60      0       41      50    5    0     618   1641   
           18   79   56   68      5       51      58    0    3     643   1739   
           19   51   32   42     -4       33      40   23    0     623   1647   
           20   80   52   66     -4       50      57    0    1     623   1647   
2014-10-31 1    53   40   47     -4       36      41   18    0     623   1647   
           2    49   40   45      0       35      40   20    0     619   1643   
           3    55   33   44    -12       32      41   21    0     645   1729   
           4    76   56   66     -1       47      56    0    1     644   1745   
           5    76   56   66     -1       37      56    0    1     616   1640   
           6    76   51   64      0       47      55    1    0     644   1745   
           7    58   38   48      0       32      41   17    0     644   1745   
           8    78   55   67      0       32 

In [25]:
def get_station_data(store_nbr, store_station_pairs):
    station_nbr = store_station_pairs.loc[store_nbr].values[0]
    return station_nbr

In [26]:
def get_weather_data(date, store_nbr, store_station_pairs, weather_file):
    station_nbr = store_station_pairs.loc[store_nbr].values[0]
    weather_data = weather_file.loc[date, station_nbr].values
    return weather_data

In [27]:
def get_item_list():
    #train data에서 item_nbr data중복 제거 후 list로 변환
    train_file = pd.read_csv("Data/train.csv", sep=',')
    item_list = list(set(list(train_file.values[:,2])))
    return item_list

In [28]:
def get_item_data(item_nbr, percent, store_station_pairs, weather_file, train_file):
    start = time.time()
    train_num = train_file.values[:,1].shape[0]

    item_index = np.where(train_file.values[:,2] == item_nbr)
    item_index = item_index[0]
    num = int(item_index.shape[0]*(1-percent)) # Select according to the given percentage, need to be improved
    iter_list = range(num, item_index.shape[0])
    item_dataset = np.zeros((len(iter_list), weather_file.values.shape[1]+1))
    count = 0
    #print len(iter_list)
    for i in iter_list:
        index = item_index[i]
        #print count
        date = train_file.values[:,0][index]
        store_nbr = train_file.values[:,1][index]
        units = train_file.values[:,3][index]

        weather_data = get_weather_data(date, store_nbr, store_station_pairs, weather_file)

        item_dataset[count,:] =  np.append(weather_data, [units], axis=0)
        count += 1
    end = time.time()
    #print "Running time for item %d: %f" %(int(item_nbr), (end-start))
    return item_dataset

In [29]:
def write_item_data(tasks):
    item_list = tasks.get()
    #print item_list
    store_station_pairs = match_store_station()
    weather_file = read_processed_weather() # index has 2 columns and values has 17 columns
    train_file = pd.read_csv("Data/train.csv", sep=',')

    #item_list = get_item_list()
    #item_nbr = item_list[0]
    for item_nbr in item_list:
        item_dataset = get_item_data(item_nbr, 0.1, store_station_pairs, weather_file, train_file) # Give percentage
        df = pd.DataFrame(item_dataset, index=None, columns=None)
        filename = "Item_data/item_%d.csv" %(int(item_nbr))
        df.to_csv(filename, sep=',', index=None, columns=None)

In [30]:
def multi_process(traget_func):
    processors = multiprocessing.cpu_count()

    myTasks = multiprocessing.Queue()
    item_list = get_item_list()
    temp_part = []
    div = len(item_list)/(processors-1)
    rem = len(item_list)%(processors-1)
    ind = 0
    while ind < div*(processors-1): 
        temp_part.append(item_list[ind:ind+div])
        ind = ind+div
    temp_part.append(item_list[(len(item_list)-rem):])

    for each in temp_part:
        myTasks.put(each)

    Workers = [multiprocessing.Process(target = traget_func, args =(myTasks,)) for i in range(processors)]

    #Workers[0].start()

    for each in Workers:
        each.start()

In [31]:
def read_item_data(item_nbr):
    filename = "Item_data/item_%d.csv" %(int(item_nbr))
    item_dataset = pd.read_csv(filename, sep=',')

    item_data = item_dataset.values[:,0:-1]
    item_label = item_dataset.values[:,-1]
    return item_data, item_label

In [32]:
def split_item_data(item_nbr, type):
    item_data, item_label = read_item_data(item_nbr)
    X_train, X_test, y_train, y_test = train_test_split(item_data, item_label, test_size=0.25, random_state=42)
    if type == 'train':
        return X_train, y_train
    elif type == 'test':
        return X_test, y_test

In [33]:
train_file = pd.read_csv("Data/train.csv", sep=',')
train_file = train_file[train_file['units'] > 0]
train_file    

date  store_nbr  item_nbr  units
8        2012-01-01          1         9     29
27       2012-01-01          1        28      2
50       2012-01-01          1        51      1
115      2012-01-01          2         5    191
154      2012-01-01          2        44    215
226      2012-01-01          3         5    214
266      2012-01-01          3        45    112
341      2012-01-01          4         9     61
359      2012-01-01          4        27     21
459      2012-01-01          5        16     24
475      2012-01-01          5        32      1
480      2012-01-01          5        37     16
563      2012-01-01          6         9     44
599      2012-01-01          6        45     86
670      2012-01-01          7         5    137
672      2012-01-01          7         7      2
688      2012-01-01          7        23     16
781      2012-01-01          8         5     38
799      2012-01-01          8        23      4
892      2012-01-01          9         5    131
897      2012-01-01          9        10      4
932      2012-01-01          9        45     72
980      2012-01-01          9        93      4
1003     2012-01-01         10         5     98
1035     2012-01-01         10        37     32
1091     2012-01-01         10        93      3
1118     2012-01-01         11         9     74
1177     2012-01-01         11        68     48
1236     2012-01-01         12        16     29
1257     2012-01-01         12        37     28
...             ...        ...       ...    ...
4616394  2014-10-31         35        16     18
4616444  2014-10-31         35        66     10
4616498  2014-10-31         36         9     29
4616519  2014-10-31         36        30      2
4616533  2014-10-31         36        44     23
4616605  2014-10-31         37         5     15
4616638  2014-10-31         37        38     10
4616645  2014-10-31         37        45     20
4616653  2014-10-31         37        53      2
4616720  2014-10-31         38         9     47
4616756  2014-10-31         38        45     63
4616831  2014-10-31         39         9     12
4616858  2014-10-31         39        36      3
4616925  2014-10-31         39       103      4
4616933  2014-10-31         39       111      2
4616938  2014-10-31         40         5     18
4616978  2014-10-31         40        45     44
4617060  2014-10-31         41        16     30
4617112  2014-10-31         41        68     15
4617152  2014-10-31         41       108      7
4617160  2014-10-31         42         5     29
4617200  2014-10-31         42        45     35
4617282  2014-10-31         43        16     39
4617303  2014-10-31         43        37      8
4617339  2014-10-31         43        73      1
4617382  2014-10-31         44         5     47
4617445  2014-10-31         44        68     39
4617471  2014-10-31         44        94      1
4617497  2014-10-31         45         9     10
4617514  2014-10-31         45        26      1

[118696 rows x 4 columns]

In [34]:
def get_full_data():
    train_file = pd.read_csv("Data/train.csv", sep=',')
    train_file = train_file[train_file['units'] > 0]

    store_station_pairs = match_store_station()
    #weather_file = read_weather() # index has 2 columns and values has 18 columns
    weather_file = read_processed_weather()
    train_num = train_file.values[:,1].shape[0]
    item_dataset = [] # Should be 20 columns
    for ind in tqdm(range(train_num)):
        store_nbr = train_file.values[:,1][ind]
        date = train_file.values[:,0][ind]  
        weather_data = get_weather_data(date, store_nbr, store_station_pairs, weather_file)
        station_nbr = get_station_data(store_nbr, store_station_pairs)
        #weather_data = np.append(train_file.values[:,2:][ind], weather_data, axis = 1) #[item_nbr, units, weather_features]
        #weather_data = np.concatenate((train_file.values[:,2:][ind], weather_data), axis = 0)
        date = np.array([date, store_nbr, station_nbr])
        result = np.concatenate((date, train_file.values[:,2:][ind], weather_data), axis = 0)
        
        item_dataset.append(result)

    item_dataset = np.array(item_dataset)
    #train_columns = np.append(np.array(['item_nbr','units']), weather_file.columns.values, axis=1)
    train_columns = np.concatenate((np.array(['date', 'store_nbr', 'station_nbr', 'item_nbr','units']), weather_file.columns.values), axis=0)
    train_df = pd.DataFrame(item_dataset, index=None, columns=train_columns) # Build item_nbr index on units and weather features
    train_df.to_csv("train_feature_matrix.csv", sep=',', index=None)
    return train_df

In [35]:
get_full_data()

  0%|          | 32/4498904 [00:29<1164:57:22,  1.07it/s]

KeyboardInterrupt: 

In [65]:
train_features = pd.read_csv("train_feature_matrix.csv", sep=',')
holidays = pd.read_csv('holidays.csv', sep=',')

In [66]:
train_features.tail()

date  store_nbr  station_nbr  item_nbr  units  tmax  tmin  tavg  \
118691  2014-10-31         44           12         5     47  75.0  60.0  68.0   
118692  2014-10-31         44           12        68     39  75.0  60.0  68.0   
118693  2014-10-31         44           12        94      1  75.0  60.0  68.0   
118694  2014-10-31         45           16         9     10  53.0  34.0  44.0   
118695  2014-10-31         45           16        26      1  53.0  34.0  44.0   

        depart  dewpoint    ...     cool  sunrise  sunset  snowfall  \
118691    -2.0      53.0    ...      3.0    640.0  1740.0       0.0   
118692    -2.0      53.0    ...      3.0    640.0  1740.0       0.0   
118693    -2.0      53.0    ...      3.0    640.0  1740.0       0.0   
118694    -2.0      35.0    ...      0.0    619.0  1640.0       0.0   
118695    -2.0      35.0    ...      0.0    619.0  1640.0       0.0   

        preciptotal  stnpressure  sealevel  resultspeed  resultdir  avgspeed  
118691          0.0         29.8     29.85          7.0       32.0       7.5  
118692          0.0         29.8     29.85          7.0       32.0       7.5  
118693          0.0         29.8     29.85          7.0       32.0       7.5  
118694          0.0         29.9     29.99          4.5        3.0       5.3  
118695          0.0         29.9     29.99          4.5        3.0       5.3  

[5 rows x 22 columns]

In [67]:
holidays.tail()

date     day     holiday_name
41  2014-06-15  Sunday       FathersDay
42  2014-07-04  Friday  IndependenceDay
43  2014-09-01  Monday         LaborDay
44  2014-10-13  Monday      ColumbusDay
45  2014-10-31  Friday        Halloween

In [68]:
tmp_result = pd.merge(train_features, holidays, left_on='date', right_on='date', how='outer')
tmp_result.to_csv("train_holidays_merge_matrix.csv", sep=',', index=None)

In [11]:
result = pd.read_csv('train_holidays_merge_matrix.csv', sep=',')
result

date  store_nbr  station_nbr  item_nbr  units  tmax  tmin  tavg  \
0       2012-01-01        1.0          1.0       9.0   29.0  52.0  31.0  42.0   
1       2012-01-01        1.0          1.0      28.0    2.0  52.0  31.0  42.0   
2       2012-01-01        1.0          1.0      51.0    1.0  52.0  31.0  42.0   
3       2012-01-01        2.0         14.0       5.0  191.0  50.0  34.0  42.0   
4       2012-01-01        2.0         14.0      44.0  215.0  50.0  34.0  42.0   
5       2012-01-01        3.0          7.0       5.0  214.0  50.0  33.0  42.0   
6       2012-01-01        3.0          7.0      45.0  112.0  50.0  33.0  42.0   
7       2012-01-01        4.0          9.0       9.0   61.0  34.0  19.0  27.0   
8       2012-01-01        4.0          9.0      27.0   21.0  34.0  19.0  27.0   
9       2012-01-01        5.0         12.0      16.0   24.0  72.0  48.0  60.0   
10      2012-01-01        5.0         12.0      32.0    1.0  72.0  48.0  60.0   
11      2012-01-01        5.0         12.0      37.0   16.0  72.0  48.0  60.0   
12      2012-01-01        6.0         14.0       9.0   44.0  50.0  34.0  42.0   
13      2012-01-01        6.0         14.0      45.0   86.0  50.0  34.0  42.0   
14      2012-01-01        7.0          6.0       5.0  137.0  63.0  34.0  49.0   
15      2012-01-01        7.0          6.0       7.0    2.0  63.0  34.0  49.0   
16      2012-01-01        7.0          6.0      23.0   16.0  63.0  34.0  49.0   
17      2012-01-01        8.0          4.0       5.0   38.0  63.0  47.0  55.0   
18      2012-01-01        8.0          4.0      23.0    4.0  63.0  47.0  55.0   
19      2012-01-01        9.0         17.0       5.0  131.0  56.0  38.0  47.0   
20      2012-01-01        9.0         17.0      10.0    4.0  56.0  38.0  47.0   
21      2012-01-01        9.0         17.0      45.0   72.0  56.0  38.0  47.0   
22      2012-01-01        9.0         17.0      93.0    4.0  56.0  38.0  47.0   
23      2012-01-01       10.0         12.0       5.0   98.0  72.0  48.0  60.0   
24      2012-01-01       10.0         12.0      37.0   32.0  72.0  48.0  60.0   
25      2012-01-01       10.0         12.0      93.0    3.0  72.0  48.0  60.0   
26      2012-01-01       11.0         10.0       9.0   74.0  73.0  53.0  63.0   
27      2012-01-01       11.0         10.0      68.0   48.0  73.0  53.0  63.0   
28      2012-01-01       12.0         11.0      16.0   29.0  72.0  48.0  60.0   
29      2012-01-01       12.0         11.0      37.0   28.0  72.0  48.0  60.0   
...            ...        ...          ...       ...    ...   ...   ...   ...   
118667  2014-10-31       35.0          5.0      66.0   10.0  76.0  56.0  66.0   
118668  2014-10-31       36.0         18.0       9.0   29.0  67.0  49.0  58.0   
118669  2014-10-31       36.0         18.0      30.0    2.0  67.0  49.0  58.0   
118670  2014-10-31       36.0         18.0      44.0   23.0  67.0  49.0  58.0   
118671  2014-10-31       37.0         13.0       5.0   15.0  58.0  35.0  47.0   
118672  2014-10-31       37.0         13.0      38.0   10.0  58.0  35.0  47.0   
118673  2014-10-31       37.0         13.0      45.0   20.0  58.0  35.0  47.0   
118674  2014-10-31       37.0         13.0      53.0    2.0  58.0  35.0  47.0   
118675  2014-10-31       38.0         14.0       9.0   47.0  60.0  38.0  49.0   
118676  2014-10-31       38.0         14.0      45.0   63.0  60.0  38.0  49.0   
118677  2014-10-31       39.0          8.0       9.0   12.0  78.0  55.0  67.0   
118678  2014-10-31       39.0          8.0      36.0    3.0  78.0  55.0  67.0   
118679  2014-10-31       39.0          8.0     103.0    4.0  78.0  55.0  67.0   
118680  2014-10-31       39.0          8.0     111.0    2.0  78.0  55.0  67.0   
118681  2014-10-31       40.0         13.0       5.0   18.0  58.0  35.0  47.0   
118682  2014-10-31       40.0         13.0      45.0   44.0  58.0  35.0  47.0   
118683  2014-10-31       41.0         12.0      16.0   30.0  75.0  60.0  68.0   
118684  2014-10-31       41.0         12.0   

In [23]:
weather_cs = weather_cs.drop(['tmax', 'tmin', 'tavg', 'depart', 'dewpoint',
       'wetbulb', 'heat', 'cool', 'sunrise', 'sunset', 'snowfall',
       'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir',
       'avgspeed'], axis = 1) # codesum 컬럼 삭제
weather_cs

station_nbr        date         codesum  +FC BC BL BR   DR   DS DU ...  \
0                1  2012-01-01      RA FZFG BR  0.0  0  0  1  0.0  0.0  0 ...   
1                2  2012-01-01              RA  0.0  0  0  0  0.0  0.0  0 ...   
2                3  2012-01-01                  0.0  0  0  0  0.0  0.0  0 ...   
3                4  2012-01-01                  0.0  0  0  0  0.0  0.0  0 ...   
4                6  2012-01-01                  0.0  0  0  0  0.0  0.0  0 ...   
5                7  2012-01-01                  0.0  0  0  0  0.0  0.0  0 ...   
6                8  2012-01-01           RA BR  0.0  0  0  1  0.0  0.0  0 ...   
7                9  2012-01-01              UP  0.0  0  0  0  0.0  0.0  0 ...   
8               10  2012-01-01       FG+ FG BR  0.0  0  0  1  0.0  0.0  0 ...   
9               11  2012-01-01       FG+ FG BR  0.0  0  0  1  0.0  0.0  0 ...   
10              12  2012-01-01       FG+ FG BR  0.0  0  0  1  0.0  0.0  0 ...   
11              13  2012-01-01                  0.0  0  0  0  0.0  0.0  0 ...   
12              14  2012-01-01                  0.0  0  0  0  0.0  0.0  0 ...   
13              15  2012-01-01  RA FG+ FZFG BR  0.0  0  0  1  0.0  0.0  0 ...   
14              16  2012-01-01      RA MIFG BR  0.0  0  0  1  0.0  0.0  0 ...   
15              17  2012-01-01                  0.0  0  0  0  0.0  0.0  0 ...   
16              18  2012-01-01                  0.0  0  0  0  0.0  0.0  0 ...   
17              19  2012-01-01        RA SN BR  0.0  0  0  1  0.0  0.0  0 ...   
18              20  2012-01-01                  0.0  0  0  0  0.0  0.0  0 ...   
19               1  2012-01-02                  0.0  0  0  0  0.0  0.0  0 ...   
20               2  2012-01-02                  0.0  0  0  0  0.0  0.0  0 ...   
21               3  2012-01-02                  0.0  0  0  0  0.0  0.0  0 ...   
22               4  2012-01-02                  0.0  0  0  0  0.0  0.0  0 ...   
23               6  2012-01-02                  0.0  0  0  0  0.0  0.0  0 ...   
24               7  2012-01-02                  0.0  0  0  0  0.0  0.0  0 ...   
25               8  2012-01-02                  0.0  0  0  0  0.0  0.0  0 ...   
26               9  2012-01-02           SN UP  0.0  0  0  0  0.0  0.0  0 ...   
27              10  2012-01-02                  0.0  0  0  0  0.0  0.0  0 ...   
28              11  2012-01-02                  0.0  0  0  0  0.0  0.0  0 ...   
29              12  2012-01-02                  0.0  0  0  0  0.0  0.0  0 ...   
...            ...         ...             ...  ... .. .. ..  ...  ... .. ...   
20487           11  2014-10-30           TS BR  0.0  0  0  1  0.0  0.0  0 ...   
20488           12  2014-10-30         TSRA BR  0.0  0  0  1  0.0  0.0  0 ...   
20489           13  2014-10-30                  0.0  0  0  0  0.0  0.0  0 ...   
20490           14  2014-10-30                  0.0  0  0  0  0.0  0.0  0 ...   
20491           15  2014-10-30           FG BR  0.0  0  0  1  0.0  0.0  0 ...   
20492           16  2014-10-30                  0.0  0  0  0  0.0  0.0  0 ...   
20493           17  2014-10-30                  0.0  0  0  0  0.0  0.0  0 ...   
20494           18  2014-10-30              RA  0.0  0  0  0  0.0  0.0  0 ...   
20495           19  2014-10-30              RA  0.0  0  0  0  0.0  0.0  0 ...   
20496           20  2014-10-30                  0.0  0  0  0  0.0  0.0  0 ...   
20497            1  2014-10-31                  0.0  0  0  0  0.0  0.0  0 ...   
20498            2  2014-10-31                  0.0  0  0  0  0.0  0.0  0 ...   
20499            3  2014-10-31                  0.0  0  0  0  0.0  0.0  0 ...   
20500            4  2014-10-31           RA BR  0.0  0  0  1  0.0  0.0  0 ...   
20501            5  2014-10-31                  0.0  0  0  0  0.0  0.0  0 ...   
20502            6  2014-10-31           RA BR  0.0  0  0  1  0.0  0.0  0 ...   
20503            7  2014-10-31                  0.0  0  0  0  0.0  0.0  0 ...   
20504            8  2014-10-31        

In [105]:
result = pd.merge(result, weather_cs, on=['date', 'station_nbr'])
result.to_csv("train_result_matrix.csv", sep=',', index=None)

Index(['date', 'store_nbr', 'station_nbr', 'item_nbr', 'units', 'tmax', 'tmin',
       'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise',
       'sunset', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'resultspeed', 'resultdir', 'avgspeed', 'day', 'holiday_name', '+FC',
       'BC', 'BL', 'BR', 'DR', 'DS', 'DU', 'DZ', 'FC', 'FG', 'FG+', 'FU', 'FZ',
       'GR', 'GS', 'HZ', 'IC', 'MI', 'PL', 'PO', 'PR', 'PY', 'RA', 'SA', 'SG',
       'SH', 'SN', 'SO', 'SQ', 'SS', 'TS', 'UP', 'VA', 'VC'],
      dtype='object')

In [43]:
result = pd.read_csv('train_result_matrix.csv', sep=',')
result

date  store_nbr  station_nbr  item_nbr  units  tmax  tmin  tavg  \
0       2012-01-01        1.0          1.0       9.0   29.0  52.0  31.0  42.0   
1       2012-01-01        1.0          1.0      28.0    2.0  52.0  31.0  42.0   
2       2012-01-01        1.0          1.0      51.0    1.0  52.0  31.0  42.0   
3       2012-01-01        2.0         14.0       5.0  191.0  50.0  34.0  42.0   
4       2012-01-01        2.0         14.0      44.0  215.0  50.0  34.0  42.0   
5       2012-01-01        6.0         14.0       9.0   44.0  50.0  34.0  42.0   
6       2012-01-01        6.0         14.0      45.0   86.0  50.0  34.0  42.0   
7       2012-01-01       38.0         14.0       9.0  195.0  50.0  34.0  42.0   
8       2012-01-01       38.0         14.0      45.0  135.0  50.0  34.0  42.0   
9       2012-01-01       42.0         14.0       5.0  154.0  50.0  34.0  42.0   
10      2012-01-01       42.0         14.0      45.0   98.0  50.0  34.0  42.0   
11      2012-01-01        3.0          7.0       5.0  214.0  50.0  33.0  42.0   
12      2012-01-01        3.0          7.0      45.0  112.0  50.0  33.0  42.0   
13      2012-01-01       20.0          7.0       5.0  165.0  50.0  33.0  42.0   
14      2012-01-01       20.0          7.0      41.0   37.0  50.0  33.0  42.0   
15      2012-01-01       20.0          7.0      74.0    5.0  50.0  33.0  42.0   
16      2012-01-01       28.0          7.0       5.0   65.0  50.0  33.0  42.0   
17      2012-01-01       28.0          7.0      45.0   68.0  50.0  33.0  42.0   
18      2012-01-01        4.0          9.0       9.0   61.0  34.0  19.0  27.0   
19      2012-01-01        4.0          9.0      27.0   21.0  34.0  19.0  27.0   
20      2012-01-01       24.0          9.0       6.0   42.0  34.0  19.0  27.0   
21      2012-01-01       24.0          9.0      43.0   72.0  34.0  19.0  27.0   
22      2012-01-01       24.0          9.0      60.0    1.0  34.0  19.0  27.0   
23      2012-01-01        5.0         12.0      16.0   24.0  72.0  48.0  60.0   
24      2012-01-01        5.0         12.0      32.0    1.0  72.0  48.0  60.0   
25      2012-01-01        5.0         12.0      37.0   16.0  72.0  48.0  60.0   
26      2012-01-01       10.0         12.0       5.0   98.0  72.0  48.0  60.0   
27      2012-01-01       10.0         12.0      37.0   32.0  72.0  48.0  60.0   
28      2012-01-01       10.0         12.0      93.0    3.0  72.0  48.0  60.0   
29      2012-01-01       41.0         12.0      16.0   42.0  72.0  48.0  60.0   
...            ...        ...          ...       ...    ...   ...   ...   ...   
118666  2014-10-31       40.0         13.0       5.0   18.0  58.0  35.0  47.0   
118667  2014-10-31       40.0         13.0      45.0   44.0  58.0  35.0  47.0   
118668  2014-10-31       16.0          2.0      16.0   16.0  49.0  40.0  45.0   
118669  2014-10-31       16.0          2.0      25.0  101.0  49.0  40.0  45.0   
118670  2014-10-31       16.0          2.0      64.0    3.0  49.0  40.0  45.0   
118671  2014-10-31       17.0         20.0       9.0   24.0  68.0  50.0  59.0   
118672  2014-10-31       17.0         20.0      37.0   25.0  68.0  50.0  59.0   
118673  2014-10-31       19.0         15.0      16.0   10.0  50.0  32.0  41.0   
118674  2014-10-31       19.0         15.0      75.0    2.0  50.0  32.0  41.0   
118675  2014-10-31       19.0         15.0      83.0    4.0  50.0  32.0  41.0   
118676  2014-10-31       21.0          3.0       5.0   15.0  55.0  33.0  44.0   
118677  2014-10-31       21.0          3.0      44.0   57.0  55.0  33.0  44.0   
118678  2014-10-31       21.0          3.0     109.0    7.0  55.0  33.0  44.0   
118679  2014-10-31       29.0          3.0       8.0    7.0  55.0  33.0  44.0   
118680  2014-10-31       29.0          3.0      41.0    9.0  55.0  33.0  44.0   
118681  2014-10-31       29.0          3.0      45.0   26.0  55.0  33.0  44.0   
118682  2014-10-31       33.0          3.0       9.0   73.0  55.0  33.0  44.0   
118683  2014-10-31       33.0          3.0   

In [25]:
ls = result.columns
ls

Index(['date', 'store_nbr', 'station_nbr', 'item_nbr', 'units', 'tmax', 'tmin',
       'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise',
       'sunset', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'resultspeed', 'resultdir', 'avgspeed', 'day', 'holiday_name', '+FC',
       'BC', 'BL', 'BR', 'DR', 'DS', 'DU', 'DZ', 'FC', 'FG', 'FG+', 'FU', 'FZ',
       'GR', 'GS', 'HZ', 'IC', 'MI', 'PL', 'PO', 'PR', 'PY', 'RA', 'SA', 'SG',
       'SH', 'SN', 'SO', 'SQ', 'SS', 'TS', 'UP', 'VA', 'VC'],
      dtype='object')

In [44]:
result['date'] = pd.to_datetime(result['date'], errors='coerce')
result['year'] = result['date'].dt.year
result['month'] = result['date'].dt.month
result['day'] = result['date'].dt.day
result['weekday'] = result[['date']].apply(lambda x: dt.datetime.strftime(x['date'], '%a'), axis=1)
result

date  store_nbr  station_nbr  item_nbr  units  tmax  tmin  tavg  \
0      2012-01-01        1.0          1.0       9.0   29.0  52.0  31.0  42.0   
1      2012-01-01        1.0          1.0      28.0    2.0  52.0  31.0  42.0   
2      2012-01-01        1.0          1.0      51.0    1.0  52.0  31.0  42.0   
3      2012-01-01        2.0         14.0       5.0  191.0  50.0  34.0  42.0   
4      2012-01-01        2.0         14.0      44.0  215.0  50.0  34.0  42.0   
5      2012-01-01        6.0         14.0       9.0   44.0  50.0  34.0  42.0   
6      2012-01-01        6.0         14.0      45.0   86.0  50.0  34.0  42.0   
7      2012-01-01       38.0         14.0       9.0  195.0  50.0  34.0  42.0   
8      2012-01-01       38.0         14.0      45.0  135.0  50.0  34.0  42.0   
9      2012-01-01       42.0         14.0       5.0  154.0  50.0  34.0  42.0   
10     2012-01-01       42.0         14.0      45.0   98.0  50.0  34.0  42.0   
11     2012-01-01        3.0          7.0       5.0  214.0  50.0  33.0  42.0   
12     2012-01-01        3.0          7.0      45.0  112.0  50.0  33.0  42.0   
13     2012-01-01       20.0          7.0       5.0  165.0  50.0  33.0  42.0   
14     2012-01-01       20.0          7.0      41.0   37.0  50.0  33.0  42.0   
15     2012-01-01       20.0          7.0      74.0    5.0  50.0  33.0  42.0   
16     2012-01-01       28.0          7.0       5.0   65.0  50.0  33.0  42.0   
17     2012-01-01       28.0          7.0      45.0   68.0  50.0  33.0  42.0   
18     2012-01-01        4.0          9.0       9.0   61.0  34.0  19.0  27.0   
19     2012-01-01        4.0          9.0      27.0   21.0  34.0  19.0  27.0   
20     2012-01-01       24.0          9.0       6.0   42.0  34.0  19.0  27.0   
21     2012-01-01       24.0          9.0      43.0   72.0  34.0  19.0  27.0   
22     2012-01-01       24.0          9.0      60.0    1.0  34.0  19.0  27.0   
23     2012-01-01        5.0         12.0      16.0   24.0  72.0  48.0  60.0   
24     2012-01-01        5.0         12.0      32.0    1.0  72.0  48.0  60.0   
25     2012-01-01        5.0         12.0      37.0   16.0  72.0  48.0  60.0   
26     2012-01-01       10.0         12.0       5.0   98.0  72.0  48.0  60.0   
27     2012-01-01       10.0         12.0      37.0   32.0  72.0  48.0  60.0   
28     2012-01-01       10.0         12.0      93.0    3.0  72.0  48.0  60.0   
29     2012-01-01       41.0         12.0      16.0   42.0  72.0  48.0  60.0   
...           ...        ...          ...       ...    ...   ...   ...   ...   
118666 2014-10-31       40.0         13.0       5.0   18.0  58.0  35.0  47.0   
118667 2014-10-31       40.0         13.0      45.0   44.0  58.0  35.0  47.0   
118668 2014-10-31       16.0          2.0      16.0   16.0  49.0  40.0  45.0   
118669 2014-10-31       16.0          2.0      25.0  101.0  49.0  40.0  45.0   
118670 2014-10-31       16.0          2.0      64.0    3.0  49.0  40.0  45.0   
118671 2014-10-31       17.0         20.0       9.0   24.0  68.0  50.0  59.0   
118672 2014-10-31       17.0         20.0      37.0   25.0  68.0  50.0  59.0   
118673 2014-10-31       19.0         15.0      16.0   10.0  50.0  32.0  41.0   
118674 2014-10-31       19.0         15.0      75.0    2.0  50.0  32.0  41.0   
118675 2014-10-31       19.0         15.0      83.0    4.0  50.0  32.0  41.0   
118676 2014-10-31       21.0          3.0       5.0   15.0  55.0  33.0  44.0   
118677 2014-10-31       21.0          3.0      44.0   57.0  55.0  33.0  44.0   
118678 2014-10-31       21.0          3.0     109.0    7.0  55.0  33.0  44.0   
118679 2014-10-31       29.0          3.0       8.0    7.0  55.0  33.0  44.0   
118680 2014-10-31       29.0          3.0      41.0    9.0  55.0  33.0  44.0   
118681 2014-10-31       29.0          3.0      45.0   26.0  55.0  33.0  44.0   
118682 2014-10-31       33.0          3.0       9.0   73.0  55.0  33.0  44.0   
118683 2014-10-31       33.0          3.0      44.0  264.0  55.0  33.0  44.0   
118684 2014-1

In [45]:
tmp_result = result.set_index('date')[['store_nbr', 'station_nbr', 'item_nbr', 'units', 'tmax', 'tmin',
       'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise',
       'sunset', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'resultspeed', 'resultdir', 'avgspeed', '+FC',
       'BC', 'BL', 'BR', 'DR', 'DS', 'DU', 'DZ', 'FC', 'FG', 'FG+', 'FU', 'FZ',
       'GR', 'GS', 'HZ', 'IC', 'MI', 'PL', 'PO', 'PR', 'PY', 'RA', 'SA', 'SG',
       'SH', 'SN', 'SO', 'SQ', 'SS', 'TS', 'UP', 'VA', 'VC', 'year', 'month',
       'day','weekday', 'holiday_name']]
tmp_result = tmp_result.reset_index()
tmp_result

date  store_nbr  station_nbr  item_nbr  units  tmax  tmin  tavg  \
0      2012-01-01        1.0          1.0       9.0   29.0  52.0  31.0  42.0   
1      2012-01-01        1.0          1.0      28.0    2.0  52.0  31.0  42.0   
2      2012-01-01        1.0          1.0      51.0    1.0  52.0  31.0  42.0   
3      2012-01-01        2.0         14.0       5.0  191.0  50.0  34.0  42.0   
4      2012-01-01        2.0         14.0      44.0  215.0  50.0  34.0  42.0   
5      2012-01-01        6.0         14.0       9.0   44.0  50.0  34.0  42.0   
6      2012-01-01        6.0         14.0      45.0   86.0  50.0  34.0  42.0   
7      2012-01-01       38.0         14.0       9.0  195.0  50.0  34.0  42.0   
8      2012-01-01       38.0         14.0      45.0  135.0  50.0  34.0  42.0   
9      2012-01-01       42.0         14.0       5.0  154.0  50.0  34.0  42.0   
10     2012-01-01       42.0         14.0      45.0   98.0  50.0  34.0  42.0   
11     2012-01-01        3.0          7.0       5.0  214.0  50.0  33.0  42.0   
12     2012-01-01        3.0          7.0      45.0  112.0  50.0  33.0  42.0   
13     2012-01-01       20.0          7.0       5.0  165.0  50.0  33.0  42.0   
14     2012-01-01       20.0          7.0      41.0   37.0  50.0  33.0  42.0   
15     2012-01-01       20.0          7.0      74.0    5.0  50.0  33.0  42.0   
16     2012-01-01       28.0          7.0       5.0   65.0  50.0  33.0  42.0   
17     2012-01-01       28.0          7.0      45.0   68.0  50.0  33.0  42.0   
18     2012-01-01        4.0          9.0       9.0   61.0  34.0  19.0  27.0   
19     2012-01-01        4.0          9.0      27.0   21.0  34.0  19.0  27.0   
20     2012-01-01       24.0          9.0       6.0   42.0  34.0  19.0  27.0   
21     2012-01-01       24.0          9.0      43.0   72.0  34.0  19.0  27.0   
22     2012-01-01       24.0          9.0      60.0    1.0  34.0  19.0  27.0   
23     2012-01-01        5.0         12.0      16.0   24.0  72.0  48.0  60.0   
24     2012-01-01        5.0         12.0      32.0    1.0  72.0  48.0  60.0   
25     2012-01-01        5.0         12.0      37.0   16.0  72.0  48.0  60.0   
26     2012-01-01       10.0         12.0       5.0   98.0  72.0  48.0  60.0   
27     2012-01-01       10.0         12.0      37.0   32.0  72.0  48.0  60.0   
28     2012-01-01       10.0         12.0      93.0    3.0  72.0  48.0  60.0   
29     2012-01-01       41.0         12.0      16.0   42.0  72.0  48.0  60.0   
...           ...        ...          ...       ...    ...   ...   ...   ...   
118666 2014-10-31       40.0         13.0       5.0   18.0  58.0  35.0  47.0   
118667 2014-10-31       40.0         13.0      45.0   44.0  58.0  35.0  47.0   
118668 2014-10-31       16.0          2.0      16.0   16.0  49.0  40.0  45.0   
118669 2014-10-31       16.0          2.0      25.0  101.0  49.0  40.0  45.0   
118670 2014-10-31       16.0          2.0      64.0    3.0  49.0  40.0  45.0   
118671 2014-10-31       17.0         20.0       9.0   24.0  68.0  50.0  59.0   
118672 2014-10-31       17.0         20.0      37.0   25.0  68.0  50.0  59.0   
118673 2014-10-31       19.0         15.0      16.0   10.0  50.0  32.0  41.0   
118674 2014-10-31       19.0         15.0      75.0    2.0  50.0  32.0  41.0   
118675 2014-10-31       19.0         15.0      83.0    4.0  50.0  32.0  41.0   
118676 2014-10-31       21.0          3.0       5.0   15.0  55.0  33.0  44.0   
118677 2014-10-31       21.0          3.0      44.0   57.0  55.0  33.0  44.0   
118678 2014-10-31       21.0          3.0     109.0    7.0  55.0  33.0  44.0   
118679 2014-10-31       29.0          3.0       8.0    7.0  55.0  33.0  44.0   
118680 2014-10-31       29.0          3.0      41.0    9.0  55.0  33.0  44.0   
118681 2014-10-31       29.0          3.0      45.0   26.0  55.0  33.0  44.0   
118682 2014-10-31       33.0          3.0       9.0   73.0  55.0  33.0  44.0   
118683 2014-10-31       33.0          3.0      44.0  264.0  55.0  33.0  44.0   
118684 2014-1

In [47]:
tmp_result.to_csv("train_daydata_result_matrix.csv", sep=',', index=None)


In [49]:
result = pd.read_csv('train_daydata_result_matrix.csv', sep=',')
result

date  store_nbr  station_nbr  item_nbr  units  tmax  tmin  tavg  \
0       2012-01-01        1.0          1.0       9.0   29.0  52.0  31.0  42.0   
1       2012-01-01        1.0          1.0      28.0    2.0  52.0  31.0  42.0   
2       2012-01-01        1.0          1.0      51.0    1.0  52.0  31.0  42.0   
3       2012-01-01        2.0         14.0       5.0  191.0  50.0  34.0  42.0   
4       2012-01-01        2.0         14.0      44.0  215.0  50.0  34.0  42.0   
5       2012-01-01        6.0         14.0       9.0   44.0  50.0  34.0  42.0   
6       2012-01-01        6.0         14.0      45.0   86.0  50.0  34.0  42.0   
7       2012-01-01       38.0         14.0       9.0  195.0  50.0  34.0  42.0   
8       2012-01-01       38.0         14.0      45.0  135.0  50.0  34.0  42.0   
9       2012-01-01       42.0         14.0       5.0  154.0  50.0  34.0  42.0   
10      2012-01-01       42.0         14.0      45.0   98.0  50.0  34.0  42.0   
11      2012-01-01        3.0          7.0       5.0  214.0  50.0  33.0  42.0   
12      2012-01-01        3.0          7.0      45.0  112.0  50.0  33.0  42.0   
13      2012-01-01       20.0          7.0       5.0  165.0  50.0  33.0  42.0   
14      2012-01-01       20.0          7.0      41.0   37.0  50.0  33.0  42.0   
15      2012-01-01       20.0          7.0      74.0    5.0  50.0  33.0  42.0   
16      2012-01-01       28.0          7.0       5.0   65.0  50.0  33.0  42.0   
17      2012-01-01       28.0          7.0      45.0   68.0  50.0  33.0  42.0   
18      2012-01-01        4.0          9.0       9.0   61.0  34.0  19.0  27.0   
19      2012-01-01        4.0          9.0      27.0   21.0  34.0  19.0  27.0   
20      2012-01-01       24.0          9.0       6.0   42.0  34.0  19.0  27.0   
21      2012-01-01       24.0          9.0      43.0   72.0  34.0  19.0  27.0   
22      2012-01-01       24.0          9.0      60.0    1.0  34.0  19.0  27.0   
23      2012-01-01        5.0         12.0      16.0   24.0  72.0  48.0  60.0   
24      2012-01-01        5.0         12.0      32.0    1.0  72.0  48.0  60.0   
25      2012-01-01        5.0         12.0      37.0   16.0  72.0  48.0  60.0   
26      2012-01-01       10.0         12.0       5.0   98.0  72.0  48.0  60.0   
27      2012-01-01       10.0         12.0      37.0   32.0  72.0  48.0  60.0   
28      2012-01-01       10.0         12.0      93.0    3.0  72.0  48.0  60.0   
29      2012-01-01       41.0         12.0      16.0   42.0  72.0  48.0  60.0   
...            ...        ...          ...       ...    ...   ...   ...   ...   
118666  2014-10-31       40.0         13.0       5.0   18.0  58.0  35.0  47.0   
118667  2014-10-31       40.0         13.0      45.0   44.0  58.0  35.0  47.0   
118668  2014-10-31       16.0          2.0      16.0   16.0  49.0  40.0  45.0   
118669  2014-10-31       16.0          2.0      25.0  101.0  49.0  40.0  45.0   
118670  2014-10-31       16.0          2.0      64.0    3.0  49.0  40.0  45.0   
118671  2014-10-31       17.0         20.0       9.0   24.0  68.0  50.0  59.0   
118672  2014-10-31       17.0         20.0      37.0   25.0  68.0  50.0  59.0   
118673  2014-10-31       19.0         15.0      16.0   10.0  50.0  32.0  41.0   
118674  2014-10-31       19.0         15.0      75.0    2.0  50.0  32.0  41.0   
118675  2014-10-31       19.0         15.0      83.0    4.0  50.0  32.0  41.0   
118676  2014-10-31       21.0          3.0       5.0   15.0  55.0  33.0  44.0   
118677  2014-10-31       21.0          3.0      44.0   57.0  55.0  33.0  44.0   
118678  2014-10-31       21.0          3.0     109.0    7.0  55.0  33.0  44.0   
118679  2014-10-31       29.0          3.0       8.0    7.0  55.0  33.0  44.0   
118680  2014-10-31       29.0          3.0      41.0    9.0  55.0  33.0  44.0   
118681  2014-10-31       29.0          3.0      45.0   26.0  55.0  33.0  44.0   
118682  2014-10-31       33.0          3.0       9.0   73.0  55.0  33.0  44.0   
118683  2014-10-31       33.0          3.0   

In [54]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118696 entries, 0 to 118695
Data columns (total 61 columns):
date            118696 non-null object
store_nbr       118696 non-null float64
station_nbr     118696 non-null float64
item_nbr        118696 non-null float64
units           118696 non-null float64
tmax            118696 non-null float64
tmin            118696 non-null float64
tavg            118696 non-null float64
depart          118696 non-null float64
dewpoint        118696 non-null float64
wetbulb         118696 non-null float64
heat            118696 non-null float64
cool            118696 non-null float64
sunrise         118696 non-null float64
sunset          118696 non-null float64
snowfall        118696 non-null float64
preciptotal     118696 non-null float64
stnpressure     118696 non-null float64
sealevel        118696 non-null float64
resultspeed     118696 non-null float64
resultdir       118696 non-null float64
avgspeed        118696 non-null float64
+FC       

In [50]:
def decision_tree(_item_nbrs, _result):
    tmp = []
    count = 0
    for idx, item  in enumerate(_item_nbrs) :
        test = result[result.item_nbr == item]
        num_attr = ['units', 'tmax', 'tmin',
       'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise',
       'sunset', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'resultspeed', 'resultdir', 'avgspeed', '+FC',
       'BC', 'BL', 'BR', 'DR', 'DS', 'DU', 'DZ', 'FC', 'FG', 'FG+', 'FU', 'FZ',
       'GR', 'GS', 'HZ', 'IC', 'MI', 'PL', 'PO', 'PR', 'PY', 'RA', 'SA', 'SG',
       'SH', 'SN', 'SO', 'SQ', 'SS', 'TS', 'UP', 'VA', 'VC']

        test = test[num_attr]

        test = test.dropna()
        n_df = pd.DataFrame(preprocessing.normalize(test), columns=num_attr)

        valueArray = test.values

        NewValueArray = n_df.values

        X = NewValueArray[:, 0:13]
        Y = valueArray[:, 0:13]

        test_size = 0.1
        seed = 5

        X_train, X_validate, Y_train, Y_validate = train_test_split(X, Y, test_size=test_size, random_state=seed)

        regr = DecisionTreeRegressor(max_depth=5, min_samples_split = 2)
        regr.fit(X_train, Y_train)

        y = regr.predict(X_validate)
        tmp.append(np.array(y))
    return tmp

In [56]:
# list_columns = ['item_nbr', 'units', 'tmax', 'tmin',
#        'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise',
#        'sunset', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
#        'resultspeed', 'resultdir', 'avgspeed', '+FC',
#        'BC', 'BL', 'BR', 'DR', 'DS', 'DU', 'DZ', 'FC', 'FG', 'FG+', 'FU', 'FZ',
#        'GR', 'GS', 'HZ', 'IC', 'MI', 'PL', 'PO', 'PR', 'PY', 'RA', 'SA', 'SG',
#        'SH', 'SN', 'SO', 'SQ', 'SS', 'TS', 'UP', 'VA', 'VC']
list_columns = ['item_nbr', 'units', 'tmax', 'tmin',
       'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise',
       'sunset', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'resultspeed', 'resultdir', 'avgspeed', 'year', 'month', 'day']

df_item = pd.DataFrame(result, columns=list_columns)

dfX = sm.add_constant(df_item)
dfy = result.units

model_boston2 = sm.OLS(dfy, dfX)
result_boston2 = model_boston2.fit()
print(result_boston2.summary())

                            OLS Regression Results                            
Dep. Variable:                  units   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.209e+28
Date:                Fri, 09 Mar 2018   Prob (F-statistic):               0.00
Time:                        10:45:19   Log-Likelihood:             2.7313e+06
No. Observations:              118696   AIC:                        -5.463e+06
Df Residuals:                  118674   BIC:                        -5.462e+06
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -4.025e-14   1.35e-13     -0.297      

In [15]:
num_attr = ['item_nbr', 'units', 'tmax', 'tmin',
       'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise',
       'sunset', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'resultspeed', 'resultdir', 'avgspeed', '+FC',
       'BC', 'BL', 'BR', 'DR', 'DS', 'DU', 'DZ', 'FC', 'FG', 'FG+', 'FU', 'FZ',
       'GR', 'GS', 'HZ', 'IC', 'MI', 'PL', 'PO', 'PR', 'PY', 'RA', 'SA', 'SG',
       'SH', 'SN', 'SO', 'SQ', 'SS', 'TS', 'UP', 'VA', 'VC']
item_nbrs = list(set(result.item_nbr.dropna()))
a = decision_tree(item_nbrs, result)


list_columns = ['item_nbr', 'units', 'tmax', 'tmin',
       'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise',
       'sunset', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'resultspeed', 'resultdir', 'avgspeed', '+FC',
       'BC', 'BL', 'BR', 'DR', 'DS', 'DU', 'DZ', 'FC', 'FG', 'FG+', 'FU', 'FZ',
       'GR', 'GS', 'HZ', 'IC', 'MI', 'PL', 'PO', 'PR', 'PY', 'RA', 'SA', 'SG',
       'SH', 'SN', 'SO', 'SQ', 'SS', 'TS', 'UP', 'VA', 'VC']


In [8]:
list_columns = ['item_nbr', 'units', 'tmax', 'tmin',
       'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise',
       'sunset', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'resultspeed', 'resultdir', 'avgspeed']

df_item = pd.DataFrame(result, columns=list_columns)
#dfX0 = pd.DataFrame(boston.data, columns=boston.feature_names)

dfX = sm.add_constant(df_item)
dfy = result.units

model_boston2 = sm.OLS(dfy, dfX)
result_boston2 = model_boston2.fit()
print(result_boston2.summary())

                            OLS Regression Results                            
Dep. Variable:                  units   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.377e+31
Date:                Wed, 07 Mar 2018   Prob (F-statistic):               0.00
Time:                        16:22:31   Log-Likelihood:             3.1041e+06
No. Observations:              118696   AIC:                        -6.208e+06
Df Residuals:                  118677   BIC:                        -6.208e+06
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -2.283e-16   4.27e-16     -0.535      

In [9]:
df_item = pd.DataFrame(result, columns=list_columns)
df_item = pd.DataFrame(preprocessing.normalize(df_item), columns=list_columns)
dfX = sm.add_constant(df_item)
dfy = result.units

model_boston2 = sm.OLS(dfy, dfX)
result_boston2 = model_boston2.fit()
print(result_boston2.summary())

                            OLS Regression Results                            
Dep. Variable:                  units   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                 2.654e+05
Date:                Wed, 07 Mar 2018   Prob (F-statistic):               0.00
Time:                        16:22:33   Log-Likelihood:            -4.0540e+05
No. Observations:              118696   AIC:                         8.108e+05
Df Residuals:                  118676   BIC:                         8.110e+05
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        3350.3995      7.643    438.358      

In [124]:

store39 = result[result['store_nbr'] == 39]
df_item = pd.DataFrame(store39, columns=['store_nbr'])
#df_item = pd.DataFrame(preprocessing.normalize(df_item), columns=list_columns)
dfX = sm.add_constant(df_item)
dfy = store39.units

model_boston2 = sm.OLS(dfy, dfX)
result_boston2 = model_boston2.fit()

influence = result_boston2.get_influence()
hat = influence.hat_matrix_diag

hat.sum()

1.0000000000000002

In [140]:
list_columns = ['item_nbr', 'units', 'tmax', 'tmin',
       'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise',
       'sunset', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'resultspeed', 'resultdir', 'avgspeed', '+FC',
       'BC', 'BL', 'BR', 'DR', 'DS', 'DU', 'DZ', 'FC', 'FG', 'FG+', 'FU', 'FZ',
       'GR', 'GS', 'HZ', 'IC', 'MI', 'PL', 'PO', 'PR', 'PY', 'RA', 'SA', 'SG',
       'SH', 'SN', 'SO', 'SQ', 'SS', 'TS', 'UP', 'VA', 'VC']
list_store_nbr = list(set(result['store_nbr']))
df = pd.DataFrame()
for columns in list_columns:
    for store_nbr in list_store_nbr:
        store = result[result['store_nbr'] == store_nbr]
        df_item = pd.DataFrame(store, columns=[columns])
        #df_item = pd.DataFrame(preprocessing.normalize(df_item), columns=list_columns)
        dfX = sm.add_constant(df_item)
        dfy = store.units

        model_boston2 = sm.OLS(dfy, dfX)
        result_boston2 = model_boston2.fit()
        f_value = result_boston2.f_pvalue
        f_test = result_boston2.f_test
        df = df.append({'store_nbr': store_nbr,
                        'f_value' : f_value,
                        'columns' : columns},
                        ignore_index=True)
df

/Users/woongkook/anaconda3/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1396: RuntimeWarning: divide by zero encountered in double_scalars
  return self.ess/self.df_model
/Users/woongkook/anaconda3/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1396: RuntimeWarning: invalid value encountered in double_scalars
  return self.ess/self.df_model


columns        f_value  store_nbr
0     item_nbr  2.655586e-146        1.0
1     item_nbr   8.184520e-11        2.0
2     item_nbr   1.025100e-73        3.0
3     item_nbr  2.007213e-316        4.0
4     item_nbr   1.500572e-71        5.0
5     item_nbr   3.457779e-48        6.0
6     item_nbr   1.439188e-20        7.0
7     item_nbr  8.742952e-209        8.0
8     item_nbr   1.244830e-04        9.0
9     item_nbr  3.363692e-178       10.0
10    item_nbr  6.567274e-121       11.0
11    item_nbr   0.000000e+00       12.0
12    item_nbr  2.846927e-124       13.0
13    item_nbr   8.400477e-82       14.0
14    item_nbr   3.966572e-17       15.0
15    item_nbr  2.417788e-119       16.0
16    item_nbr  8.493682e-137       17.0
17    item_nbr  1.581503e-240       18.0
18    item_nbr  1.053311e-204       19.0
19    item_nbr  1.760423e-200       20.0
20    item_nbr  1.547361e-114       21.0
21    item_nbr  9.787541e-140       22.0
22    item_nbr   1.340608e-21       23.0
23    item_nbr  3.580594e-122       24.0
24    item_nbr  1.328321e-160       25.0
25    item_nbr   3.153888e-51       26.0
26    item_nbr   6.334885e-18       27.0
27    item_nbr   2.114663e-32       28.0
28    item_nbr   7.111808e-04       29.0
29    item_nbr   2.384411e-20       30.0
...        ...            ...        ...
2355        VC   2.565443e-01       16.0
2356        VC   6.241998e-01       17.0
2357        VC   4.516666e-01       18.0
2358        VC   9.620605e-02       19.0
2359        VC   4.892235e-01       20.0
2360        VC   3.526274e-01       21.0
2361        VC   8.758470e-01       22.0
2362        VC   8.733606e-01       23.0
2363        VC   8.955847e-01       24.0
2364        VC   7.104298e-01       25.0
2365        VC   9.268300e-01       26.0
2366        VC   5.443183e-01       27.0
2367        VC   6.104577e-01       28.0
2368        VC   5.759520e-01       29.0
2369        VC            NaN       30.0
2370        VC   6.493736e-01       31.0
2371        VC   3.438873e-01       32.0
2372        VC   8.416418e-01       33.0
2373        VC   3.167047e-01       34.0
2374        VC            NaN       35.0
2375        VC   9.564360e-01       36.0
2376        VC   8.617851e-01       37.0
2377        VC            NaN       38.0
2378        VC   9.590258e-01       39.0
2379        VC   2.881609e-01       40.0
2380        VC   4.066219e-02       41.0
2381        VC            NaN       42.0
2382        VC   7.986362e-01       43.0
2383        VC   9.797775e-01       44.0
2384        VC   8.656964e-01       45.0

[2385 rows x 3 columns]

In [128]:

list_columns = ['item_nbr', 'units', 'tmax', 'tmin',
       'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise',
       'sunset', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'resultspeed', 'resultdir', 'avgspeed', '+FC',
       'BC', 'BL', 'BR', 'DR', 'DS', 'DU', 'DZ', 'FC', 'FG', 'FG+', 'FU', 'FZ',
       'GR', 'GS', 'HZ', 'IC', 'MI', 'PL', 'PO', 'PR', 'PY', 'RA', 'SA', 'SG',
       'SH', 'SN', 'SO', 'SQ', 'SS', 'TS', 'UP', 'VA', 'VC']
df = pd.DataFrame()
for columns in list_columns:
    store39 = result[result['store_nbr'] == 39]
    df_item = pd.DataFrame(store39, columns=[columns])
    #df_item = pd.DataFrame(preprocessing.normalize(df_item), columns=list_columns)
    dfX = sm.add_constant(df_item)
    dfy = store39.units

    model_boston2 = sm.OLS(dfy, dfX)
    result_boston2 = model_boston2.fit()
    result_boston2.summary()

/Users/woongkook/anaconda3/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1471: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/Users/woongkook/anaconda3/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1396: RuntimeWarning: divide by zero encountered in double_scalars
  return self.ess/self.df_model
/Users/woongkook/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1036: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/Users/woongkook/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/woongkook/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/woongkook/anaconda3/lib/python3.6/site-packages/sci

In [131]:
store39 = result[result['store_nbr'] == 39]
df_item = pd.DataFrame(store39, columns=list_columns)
#df_item = pd.DataFrame(preprocessing.normalize(df_item), columns=list_columns)
dfX = sm.add_constant(df_item)
dfy = store39.units

model_boston2 = sm.OLS(dfy, dfX)
result_boston2 = model_boston2.fit()
result_boston2.summary()

/Users/woongkook/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1036: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/Users/woongkook/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/woongkook/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/woongkook/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  units   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 7.110e+27
Date:                Wed, 07 Mar 2018   Prob (F-statistic):               0.00
Time:                        21:39:16   Log-Likelihood:                 53089.
No. Observations:                1982   AIC:                        -1.061e+05
Df Residuals:                    1954   BIC:                        -1.060e+05
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -8.257e-16   2.48e-15     -0.333      0.739   -5.69e-15    4.04e-15
item_nbr    -1.249e-16   4.61e-16     -0.271      0.786   -1.03e-15    7.79e-16
units           1.0000   2.34e-15   4.28e+14      0.000       1.000       1.000
tmax        -1.089e-15   2.84e-15     -0.384      0.701   -6.65e-15    4.47e-15
tmin         2.727e-15   3.25e-15      0.839      0.402   -3.65e-15     9.1e-15
tavg        -1.699e-14   5.36e-14     -0.317      0.751   -1.22e-13    8.81e-14
depart      -2.127e-15   3.07e-15     -0.694      0.488   -8.14e-15    3.89e-15
dewpoint     2.789e-15   4.72e-15      0.591      0.555   -6.47e-15    1.21e-14
wetbulb     -2.859e-15   6.44e-15     -0.444      0.657   -1.55e-14    9.77e-15
heat        -1.948e-14   5.43e-14     -0.359      0.720   -1.26e-13     8.7e-14
cool         1.737e-14   5.33e-14      0.326      0.745   -8.72e-14    1.22e-13
sunrise      1.066e-15   6.01e-16      1.774      0.076   -1.12e-16    2.24e-15
sunset       8.826e-16   3.86e-16      2.289      0.022    1.26e-16    1.64e-15
snowfall    -1.515e-16    8.4e-28   -1.8e+11      0.000   -1.51e-16   -1.51e-16
preciptotal -1.243e-14   7.28e-14     -0.171      0.864   -1.55e-13     1.3e-13
stnpressure -5.107e-15   1.07e-13     -0.048      0.962   -2.16e-13    2.05e-13
sealevel    -1.676e-14   1.13e-13     -0.149      0.882   -2.38e-13    2.04e-13
resultspeed  -4.61e-16   8.34e-15     -0.055      0.956   -1.68e-14    1.59e-14
resultdir    4.736e-16   1.54e-15      0.308      0.758   -2.54e-15    3.49e-15
avgspeed    -1.004e-15   9.06e-15     -0.111      0.912   -1.88e-14    1.68e-14
+FC            2.9e-19   2.39e-29   1.21e+10      0.000     2.9e-19     2.9e-19
BC          -5.994e-19   8.92e-29  -6.72e+09      0.000   -5.99e-19   -5.99e-19
BL            3.93e-19   7.46e-29   5.27e+09      0.000    3.93e-19    3.93e-19
BR          -1.804e-16   2.96e-14     -0.006      0.995   -5.82e-14    5.78e-14
DR           1.071e-19    3.1e-29   3.45e+09      0.000    1.07e-19    1.07e-19
DS           2.264e-19   4.04e-29    5.6e+09      0.000    2.26e-19    2.26e-19
DU           1.757e-19   2.66e-29   6.61e+09      0.000    1.76e-19    1.76e-19
DZ           6.661e-16   4.32e-14      0.015      0.988   -8.41e-14    8.54e-14
FC                   0          0        nan        nan           0           0
FG           2.109e-15    7.6e-14      0.028      0.978   -1.47e-13    1.51e-13
FG+         -5.551e-16   6.32e-14     -0.009      0.993   -1.24e-13    1.23e-13
FU                   0          0        nan        nan           0           0
FZ                   0          0        nan        nan           0           0
GR                   0          0        nan        nan           0           0
GS                   0          0        nan        nan           0           0
HZ          -6.939e-16   3.29e-14     -0.02